Here we will compare different samplig methods for the target `donut_target`, specifically the ess method 

### Importing the needed libraries 

In [1]:
from cuqi.distribution import DistributionGallery, Gaussian, JointDistribution
from cuqi.testproblem import Poisson1D
from cuqi.problem import BayesianProblem
import cuqi
import inspect
import numpy as np
import matplotlib.pyplot as plt
from cuqi.sampler import MH, CWMH, ULA, MALA, NUTS
import time
import scipy.stats as sps
from scipy.stats import gaussian_kde
import pandas as pd
import cProfile, pstats, io
from pstats import SortKey
import TableAutomization as TA

### The donut distribution


Out of all of the sampling methods we choose 5:
- MH with fixed samlpler
- MH with adapted sampler
- ULA
- MALA 
- NUTSA

We create a general sampling function, that given certain inputs will compute the according MCMC sampling

In [2]:
# The donut distribution 
target_donut = DistributionGallery("donut")


In [3]:
TA.precompute_samples(target_donut, 0.05, 100,2, x0 = np.array([0,0]), seed = 12 )

Sample 102 / 102

Average acceptance rate: 0.61 

         24010 function calls (23999 primitive calls) in 0.177 seconds

   Ordered by: primitive call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1733    0.003    0.000    0.004    0.000 {built-in method builtins.getattr}
     1634    0.002    0.000    0.002    0.000 {built-in method builtins.len}
     1184    0.001    0.000    0.001    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:328(grid)
  977/975    0.001    0.000    0.001    0.000 {built-in method builtins.isinstance}
  867/858    0.001    0.000    0.002    0.000 {built-in method builtins.hasattr}
      608    0.010    0.000    0.010    0.000 {method 'reduce' of 'numpy.ufunc' objects}
      593    0.002    0.000    0.003    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:387(par_shape)
      505    0.002    0.000    0.003    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\array\_array.py

KeyError: 0

In [10]:
def compute_LOGPDF(pr, s, search_string):
    
        # Check if the string is in the output and extract it if found
    lines = s.getvalue().split('\n')
    if any(search_string in line for line in lines):
        idx = [search_string in line for line in lines].index(True)
        return lines[idx].split()[0]
    else: return 0    

In [11]:
pr = cProfile.Profile()

#%% Profile the code
# Enable the profiler (start profiling)
pr.enable()

# CUQIpy code
Ns = 10000 # Number of samples
Nb = 0 # Number of burn-in samples
scale = 0.05 # Fixed step size "scale"
MH_sampler = MH(target_donut, scale=scale, x0=np.array([0,0]))
MH_fixed_samples = MH_sampler.sample(Ns, Nb)
# Disable the profiler (stop profiling)
pr.disable()


s = io.StringIO()
sortby = SortKey.PCALLS
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())
search_string = 'distribution/_posterior.py:84(logpdf)'  # Adjust this based on the actual output

print(compute_LOGPDF(pr, s, search_string))

Sample 10000 / 10000

Average acceptance rate: 0.8643 

         2175805 function calls (2175299 primitive calls) in 8.417 seconds

   Ordered by: primitive call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
170107/170105    0.163    0.000    0.229    0.000 {built-in method builtins.getattr}
   140622    0.077    0.000    0.077    0.000 {built-in method builtins.len}
   100164    0.041    0.000    0.041    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:328(grid)
82728/82628    0.062    0.000    0.063    0.000 {built-in method builtins.isinstance}
80221/80212    0.062    0.000    0.063    0.000 {built-in method builtins.hasattr}
    59996    0.534    0.000    0.534    0.000 {method 'reduce' of 'numpy.ufunc' objects}
    50083    0.108    0.000    0.169    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:387(par_shape)
    49995    0.023    0.000    0.023    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cu

In [15]:
#%% Example of profiling code (here we use the CUQIpy package) to extract the number of calls to a specific function (here we want to extract the number of logpdf calls in the posterior object)

#%% Importing the necessary packages
import cuqi
import cProfile, pstats, io
from pstats import SortKey
def test():
    #%% Create a profile object
    pr = cProfile.Profile()

    #%% Profile the code
    # Enable the profiler (start profiling)
    pr.enable()

    # CUQIpy code
    Ns = 10000 # Number of samples
    Nb = 0 # Number of burn-in samples
    scale = 0.05 # Fixed step size "scale"
    MH_sampler = ULA(target_donut, scale=scale, x0=np.array([0,0]))
    MH_fixed_samples = MH_sampler.sample(Ns, Nb)
    # Disable the profiler (stop profiling)
    pr.disable()

    #%% Print the profiling results
    s = io.StringIO()
    sortby = SortKey.PCALLS
    ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
    ps.print_stats()
    print(s.getvalue())

    #%% Extract the number of calls to the function we are interested in

    # extract row with string /Users/amal/Documents/research_code/CUQI-DTU/CUQIpy/cuqi/distribution/_posterior.py:84(logpdf)
    lines = s.getvalue().split('\n')
    idx = ['distribution/_posterior.py:84(logpdf)' in line for line in lines].index(False)

    # print("Index")
    # print(idx)
    # print("Line")
    # lines[idx]
    # print("Number of calls")
    return (lines[idx].split()[0])


In [16]:
test()

Sample 10000 / 10000
         4559416 function calls (4478821 primitive calls) in 10.135 seconds

   Ordered by: primitive call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
1413432/1413302    0.609    0.000    0.677    0.000 {built-in method builtins.isinstance}
   380034    0.149    0.000    0.149    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:328(grid)
   360100    0.193    0.000    0.193    0.000 {built-in method builtins.getattr}
   280673    0.125    0.000    0.125    0.000 {built-in method builtins.len}
   200016    0.338    0.000    0.558    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\geometry\_geometry.py:387(par_shape)
130236/120233    0.141    0.000    0.482    0.000 {built-in method builtins.hasattr}
   119988    0.239    0.000    0.359    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\CUQIpy\cuqi\array\_array.py:57(__array_finalize__)
    90005    0.172    0.000    0.672    0.000 C:\Users\Ich\Desktop\CUQIpy 2024\

'4559416'